<a href="https://colab.research.google.com/github/deanhoperobertson/Masters-/blob/master/Thesis/Code/Bi_LSTM_CRF_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bi-LSTM-CRF Model

In [2]:
from google.colab import files
uploaded = files.upload()

In [3]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-bnmwc3k1
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-bnmwc3k1
  Stored in directory: /tmp/pip-ephem-wheel-cache-keuqwnbj/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [4]:
!pip install sklearn_crfsuite

In [5]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split

#cusotm packages
from prepro import readfile, readstring, partial_tags

#keras and tensorflow packages
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss


#evaluation
from sklearn_crfsuite.metrics import flat_classification_report,flat_f1_score,flat_precision_score

Using TensorFlow backend.


In [0]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/test.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')

#preproces the txt file
train = readstring(train)
test = readstring(test)

#create corpus
corpus = train.copy()
corpus.extend(test)

In [17]:
words = []
tags = []
for sentence in corpus:
    for word in sentence:
        words.append(word[0])
        tags.append(word[1])
        
tags = partial_tags(tags)

setlabel=list(set(tags))
setlabel.remove('O')
        
words=list(set(words))
n_words = len(words)
print("Number of words in the dataset: ", n_words)
n_tags = len(list(set(tags)))
print("Number of Labels: ", n_tags)

Number of words in the dataset:  27316
Number of Labels:  5


In [24]:
# The first entry is reserved for PAD
tag2idx = {t: i+1 for i, t in enumerate(list(set(tags)))}
tag2idx["PAD"] = 0
# Vocabulary Key:tag_index -> Value:Label/Tag
idx2tag = {i: w for w, i in tag2idx.items()}
print("The labels LOC (location) is identified by the index: {}".format(tag2idx["ORG"]))

The labels LOC (location) is identified by the index: 3


In [25]:
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding
# Vocabulary Key:token_index -> Value:word
idx2word = {i: w for w, i in word2idx.items()}
print("The word 'rejects' is identified by the index: {}".format(word2idx["rejects"]))

The word 'rejects' is identified by the index: 15737


In [26]:
#Find the maxium length of the all the sentences in the corpus
length = []
for sentence in corpus:
  length.append(len(sentence))

MAX_LEN= max(length)
print("The maxium length of sentence is:",max(length))

The maxium length of sentence is: 124


In [66]:
# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0]] for w in s] for s in train]

# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post", value=word2idx["PAD"])
X[0]

array([24726, 15737, 25706, 26895,  5544, 18106, 17542, 26160, 16205,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0], dtype=int32)

In [67]:
def get_labeler(word):
  if word =="O":
    return word
  elif "MISC" in word:
    return word[-4:]
  else:
    return word[-3:]

# Convert Tag/Label to tag_index
y = [[tag2idx[get_labeler((w[1]))] for w in s] for s in train]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])
y[0]

array([3, 5, 2, 5, 5, 5, 2, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [0]:
# One-Hot encode categorical labels
y = [to_categorical(i, num_classes=n_tags+1) for i in y]

In [0]:
#split into test and train subsets
#X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)
#X_tr.shape,np.array(y_tr).shape, X_te.shape, np.array(y_te).shape

((12636, 124), (12636, 124, 6), (1405, 124), (1405, 124, 6))

## Building LSTM-CRF Model

In [31]:
# Model definition
EMBEDDING=40

input = Input(shape=(MAX_LEN,))
model = Embedding(input_dim=n_words+2, output_dim=EMBEDDING, # n_words + 2 (PAD & UNK)
                  input_length=MAX_LEN, mask_zero=True)(input)

model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)

model = TimeDistributed(Dense(10, activation="relu"))(model)
crf = CRF(n_tags+1)  # CRF layer
out = crf(model)  # output
model = Model(input, out)

W0728 17:57:56.321574 140476686669696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0728 17:57:56.372389 140476686669696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 17:57:56.381756 140476686669696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0728 17:57:56.557868 140476686669696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0728 17:57:56.569029 

In [32]:
model.compile(optimizer="rmsprop", loss=crf_loss)#, metrics=["accuracy"])
model.summary()

W0728 17:58:01.096566 140476686669696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 124)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 124, 40)           1092720   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 124, 100)          36400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 124, 10)           1010      
_________________________________________________________________
crf_1 (CRF)                  (None, 124, 6)            114       
Total params: 1,130,244
Trainable params: 1,130,244
Non-trainable params: 0
_________________________________________________________________


In [33]:
%%time
BATCH_SIZE = 500
EPOCHS=10

history = model.fit(X, np.array(y), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1, verbose=1)

W0728 17:58:27.114678 140476686669696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 12636 samples, validate on 1405 samples
Epoch 1/10
12636/12636 [==============================] - 22s 2ms/step - loss: 33.7572 - val_loss: 28.1730
Epoch 2/10
12636/12636 [==============================] - 16s 1ms/step - loss: 33.2611 - val_loss: 27.9776
Epoch 3/10
12636/12636 [==============================] - 16s 1ms/step - loss: 33.0512 - val_loss: 27.8304
Epoch 4/10
12636/12636 [==============================] - 16s 1ms/step - loss: 32.8807 - val_loss: 27.7252
Epoch 5/10
12636/12636 [==============================] - 16s 1ms/step - loss: 32.7443 - val_loss: 27.6630
Epoch 6/10
12636/12636 [==============================] - 16s 1ms/step - loss: 32.6442 - val_loss: 27.6241
Epoch 7/10
12636/12636 [==============================] - 16s 1ms/step - loss: 32.5680 - val_loss: 27.5829
Epoch 8/10
12636/12636 [==============================] - 16s 1ms/step - loss: 32.5098 - val_loss: 27.5652
Epoch 9/10
12636/12636 [==============================] - 16s 1ms/step - loss: 32.4623 - val_lo

## Predict on Training Set

In [35]:
pred = model.predict(X, verbose=1)

14041/14041 [==============================] - 110s 8ms/step


In [36]:
# TRain Eval
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted')
print(score)

              precision    recall  f1-score   support

         LOC       0.73      0.87      0.80      8297
        MISC       0.63      0.68      0.65      4593
           O       0.99      0.99      0.99    169578
         ORG       0.87      0.67      0.76     10025
         PAD       1.00      1.00      1.00   1537463
         PER       0.90      0.95      0.93     11128

    accuracy                           1.00   1741084
   macro avg       0.85      0.86      0.85   1741084
weighted avg       1.00      1.00      1.00   1741084

0.9955490716519337


In [37]:
report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)
#F1 Score
score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)

              precision    recall  f1-score   support

         LOC       0.73      0.87      0.80      8297
        MISC       0.63      0.68      0.65      4593
         ORG       0.87      0.67      0.76     10025
         PER       0.90      0.95      0.93     11128

   micro avg       0.81      0.81      0.81     34043
   macro avg       0.78      0.79      0.78     34043
weighted avg       0.81      0.81      0.81     34043

0.8083220531525893


## Predict on Test Set

In [0]:
# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0]] for w in s] for s in test]

# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post", value=word2idx["PAD"])
X

# Convert Tag/Label to tag_index
y = [[tag2idx[get_labeler((w[1]))] for w in s] for s in test]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])

# One-Hot encode categorical labels
y = [to_categorical(i, num_classes=n_tags+1) for i in y]

In [69]:
pred = model.predict(X, verbose=1)

3453/3453 [==============================] - 28s 8ms/step


In [72]:
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)


              precision    recall  f1-score   support

         LOC       0.32      0.78      0.45      1925
        MISC       0.41      0.53      0.46       918
         ORG       0.60      0.55      0.57      2496
         PER       0.83      0.44      0.58      2773

   micro avg       0.47      0.56      0.51      8112
   macro avg       0.54      0.57      0.51      8112
weighted avg       0.59      0.56      0.53      8112

0.53240130076626
